# Organizations GTFS Status Report

In [2]:
from calitp.tables import tbl
from siuba import *
import pandas as pd

import utils

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
from IPython.display import Markdown, HTML, display_html, display

In [4]:
import altair as alt
from dla_utils import _dla_utils
from shared_utils import styleguide


/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [5]:
from calitp import query_sql, magics

In [6]:
pd.set_option("display.max_columns", 100)

In [7]:
pct_gtfs = query_sql(
    """
WITH table AS( 
    SELECT 
    caltrans_district,
    count(*) AS org_count,
    SUM(IF(complete_static_gtfs_coverage = 1, 1, 0)) AS gtfs_static_count,
    SUM(IF(complete_rt_coverage = 1, 1, 0)) AS gtfs_rt_count,
FROM cal-itp-data-infra-staging.staging_views.airtable_orgs_filtered
GROUP BY caltrans_district
ORDER BY caltrans_district
) 

SELECT
    caltrans_district,
    org_count,
    gtfs_static_count,
    gtfs_rt_count,
    100.0 * (table.gtfs_static_count / table.org_count) AS `percent_static`,
    100.0 * (table.gtfs_rt_count / table.org_count) AS `percent_rt`
FROM table 
"""
)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [8]:
pct_gtfs

,caltrans_district,org_count,gtfs_static_count,gtfs_rt_count,percent_static,percent_rt
0,None,1,0,0,0.00,0.00
1,01 - Eureka,7,7,5,100.00,71.43
2,02 - Redding,8,7,2,87.50,25.00
3,03 - Marysville,18,15,5,83.33,27.78
4,04 - Oakland,34,24,14,70.59,41.18
5,05 - San Luis Obispo,13,11,3,84.62,23.08
6,06 - Fresno,14,12,4,85.71,28.57
7,07 - Los Angeles,55,47,19,85.45,34.55
8,08 - San Bernardino,11,11,5,100.00,45.45
9,09 - Bishop,2,2,2,100.00,100.00


In [9]:
# pct_compliant = query_sql(
# """

# WITH

# complete_status AS (
#     select
#         caltrans_district,
#         count(*) AS `count`
#     from  cal-itp-data-infra-staging.staging_views.airtable_orgs_filtered
#     where (complete_static_gtfs_coverage = 1)
#     group by caltrans_district
# ),

# all_status AS (
#     select
#         caltrans_district,
#         count(*) AS `count`
#     from  cal-itp-data-infra-staging.staging_views.airtable_orgs_filtered   
#     group by caltrans_district
# )

# SELECT
#     caltrans_district,
#     all_status.count as `all_org`,
#     complete_status.count as `gtfs_compliant`,
#     100.0 * (complete_status.count / all_status.count) AS `percent`
# FROM all_status 
# join complete_status 
#     using (caltrans_district)
# ORDER BY caltrans_district
    
# """)

In [10]:
df = query_sql(
"""
WITH new_table AS (
SELECT 
    *,
        CASE 
        WHEN (fares_v2_status LIKE ('%Unknown%') 
              or fares_v2_status is null 
              or fares_v2_status LIKE ('%Blocked%') 
              or fares_v2_status LIKE ('%to Publish%') 
              or fares_v2_status LIKE ('%Need%')) then 0 
        WHEN (fares_v2_status LIKE ('%published%')) then 1
    end AS `fares_v2_status_complete`,
        CASE 
        WHEN (fares_v2_status LIKE ('%Unknown%') or fares_v2_status is null) then 'Unknown'
        WHEN (fares_v2_status LIKE ('%Blocked%') or fares_v2_status LIKE ('%Need%')) then 'Needed'
        WHEN (fares_v2_status LIKE ('%to Publish%') or fares_v2_status LIKE ('%in progress%')) then 'In works'
        WHEN (fares_v2_status LIKE ('%published%')) then 'Published'
    end AS `fares_v2_status2`,
FROM cal-itp-data-infra-staging.staging_views.airtable_orgs_filtered
)

SELECT 
    *,
FROM new_table
""")

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [11]:
df

,organization_id,name,mobility_services_managed,parent_organization,organization_type,headquarters_place,caltrans_district,planning_authority,tracking_category,reporting_category,assist_category,headquarters_state_country,record_creation_time,service_type__from_mobility_services_managed_,currently_operating__from_mobility_services_managed_,funding_sources_for_managed_transportation,num_fixed_route_services_w_static_gtfs,at_least_one_gtfs_feed_for_any_service,service_availability_category__from_mobility_services_managed_,num_fixed_route_or_deviated_fixed_route_services,num_fixed_route_or_deviated_fixed_route_service_w__static_gtfs,num_services_with_missing_static_feed_for_fixed_route_or_deviated_fixed_route,num_fixed_route_services,complete_static_gtfs_coverage,gtfs_static_status,num_services_w_complete_rt_status,complete_rt_coverage,at_least_one_complete_rt_set,gtfs_realtime_status,missing_static,services_needing_alerts,services_needing_tripupdates_or_vehiclepositions,service_availability,fixed_route_service_operator_type,schedule_datasets,itp_id,gtfs_datasets_produced,mobility_services_operated,ntp_id,mpo_rtpa,gtfs_datasets_referenced,gtfs_dataset__from_mobility_services_managed_,total_voms__ntd_,service_area_population__ntd_,service_area_sq_miles__ntd_,currently_operating__from_mobility_services_operated_,service_type__from_mobility_services_operated_,gtfs_schedule_status,flex_status,immediate_gtfs_goals,fares_v2_status,details,website,funding_programs,county_geography,fare_systems,drmt_organization_name,drmt_reported_5310_vehicles,opm_id_drmt,alias,service___component,brand,roles,county_geography_3,dotid,administrating_organization,service_area_type__from_services_,avl_rti_vendor__from_service_area_type__from_services__,currently_operating__from_service_area_type__from_services__,services,eligibility_programs,fares_v2_status_complete,fares_v2_status2
0,recC5MAOasVIbzpAV,City of Artesia,"['recwVQU6GFZkfDnSx', 'rectIZqFrrnttOXXX']",None,City/Town,Artesia,07 - Los Angeles,['recuOdDxg9NGNH79X'],Active,Core,White Glove,CA,2021-07-17T00:45:50.000Z,"['ADA paratransit', 'fixed-route']","[True, True]","['public', 'public']",0,0,"['Public\n', 'Public\n']",1,0,1,1,0,Static Incomplete,0,0,0,RT Incomplete,['Artesia Transit'],"['Artesia Express Dial-A-Ride', 'Artesia Trans...","['Artesia Express Dial-A-Ride', 'Artesia Trans...","['Public\n', 'Public\n']",['provider-operated'],0,20.00,None,"['recwVQU6GFZkfDnSx', 'rectIZqFrrnttOXXX']",['reclLTNKOF5lIBJbH'],['recuOdDxg9NGNH79X'],None,None,[5],[12150996],[1736],"[True, True]","['ADA paratransit', 'fixed-route']","['needed', 'needed']",['Needed - Public service'],['0 - Does not have GTFS Schedule'],None,None,None,None,None,['recZ0bJA1WDvrt1yL'],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.00,Unknown
1,recV6WAkk4yeeVZLh,City of Maywood,"['recnLFaMjjcTXyAmN', 'recO2LpNVdWZV6Emn']",None,City/Town,Maywood,07 - Los Angeles,['recuOdDxg9NGNH79X'],Active,Core,White Glove,CA,2021-07-17T00:45:50.000Z,"['ADA paratransit', 'fixed-route']","[True, True]","['public', 'public']",0,0,"['Conditional', 'Public\n']",1,0,1,1,0,Static Incomplete,0,0,0,RT Incomplete,['Maywood Express Shuttle'],"['Maywood Dial-A-Ride', 'Maywood Express Shutt...","['Maywood Dial-A-Ride', 'Maywood Express Shutt...","['Conditional', 'Public\n']",['provider-operated'],0,196.00,None,"['recnLFaMjjcTXyAmN', 'recO2LpNVdWZV6Emn']",['recpjozvZELW1Neeh'],['recuOdDxg9NGNH79X'],None,None,[5],[12150996],[1736],"[True, True]","['ADA paratransit', 'fixed-route']","['needed', 'needed']",['Needed - Special service'],['0 - Does not have GTFS Schedule'],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.00,Unknown
2,recXYM27Lts8XF98x,City of Cerritos,"['recQ975tFNFBKgcL3', 'recAfsm7PMFRhTT43']",None,City/Town,Cerritos,07 - Los Angeles,['recuOdDxg9NGNH79X'],Active,Core,White Glove,CA,2021-07-17T00:45:50.000Z,"['fixed-route', 'ADA paratransit']","[True, True]","['pub

In [26]:
pd.set_option("display.max_colwidth", 1000)


In [27]:
%%sql
SELECT 
    name,
    caltrans_district,
    fares_v2_status,
    -- number of array items with 'published' case insensitive
    CAST(
          (LENGTH(fares_v2_status)-LENGTH(REPLACE(LOWER(fares_v2_status),'published',''))
          ) / LENGTH('published') AS INT64
        )                                                               AS streams_published,
    -- number of array items
    LENGTH(fares_v2_status)-LENGTH(REPLACE(fares_v2_status,',',''))+1   AS streams_total,
    -- are any streams complete?
        CASE 
        WHEN (fares_v2_status LIKE ('%Unknown%') 
              or fares_v2_status is null 
              or fares_v2_status LIKE ('%Blocked%') 
              or fares_v2_status LIKE ('% to Publish%') 
              or fares_v2_status LIKE ('%Need%')) then 0 
        WHEN (fares_v2_status LIKE ('%published%')) then 1
    end                                                                 AS `fares_v2_status_complete`,
FROM cal-itp-data-infra-staging.staging_views.airtable_orgs_filtered

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


,name,caltrans_district,fares_v2_status,streams_published,streams_total,fares_v2_status_complete
0,City of Artesia,07 - Los Angeles,None,NaN,NaN,0.00
1,City of Maywood,07 - Los Angeles,None,NaN,NaN,0.00
2,City of Cerritos,07 - Los Angeles,None,NaN,NaN,0.00
3,City of Shafter,06 - Fresno,None,NaN,NaN,0.00
4,Colusa County Transit Agency,03 - Marysville,None,NaN,NaN,0.00
...,...,...,...,...,...,...
184,Alameda-Contra Costa Transit District,04 - Oakland,"['Provider published', 'Vendor published', 'Provider published', 'Provider published']",4.00,4.00,1.00
185,San Joaquin Regional Transit District,10 - Stockton,"['Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish']",0.00,16.00,0.00
186,Eastern Sierra Transit Authority,09 - Bishop,"['Vendor published', 'Vendor published', 'Vendor published', 'Vendor published', 'Vendor published']",5.00,5.00,1.00
187,Sacramento Regional Transit District,03 - Marysville,"['Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish']",0.00,25.00,0.00


In [33]:
%%sql

WITH new_table AS (
SELECT 
    name,
    caltrans_district,
    fares_v2_status,
    SPLIT(TRIM(fares_v2_status, '[]\''), '\', \'') AS `fares_v2_status_array`
    # CAST(
    #       (LENGTH(fares_v2_status)-LENGTH(REPLACE(LOWER(fares_v2_status),'published',''))
    #       ) / LENGTH('published') AS INT64
    #     )                                                               AS streams_published,
    # -- number of array items
    # LENGTH(fares_v2_status)-LENGTH(REPLACE(fares_v2_status,',',''))+1   AS streams_total,
    # -- are any streams complete?
    #     CASE 
    #     WHEN (fares_v2_status LIKE ('%Unknown%') 
    #           or fares_v2_status is null 
    #           or fares_v2_status LIKE ('%Blocked%') 
    #           or fares_v2_status LIKE ('% to Publish%') 
    #           or fares_v2_status LIKE ('%Need%')) then 0 
    #     WHEN (fares_v2_status LIKE ('%published%')) then 1
    # end                                                                 AS `fares_v2_status_complete`,
FROM cal-itp-data-infra-staging.staging_views.airtable_orgs_filtered
)

SELECT *,
    (SELECT COUNT (*) 
     FROM UNNEST(fares_v2_status_array) AS `count_test`
     WHERE count_test LIKE ('%published%')) AS `n_published`,
    ARRAY(SELECT DISTINCT x
    FROM UNNEST(fares_v2_status_array) AS x) AS `fares_v2_distinct_status`
FROM new_table

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


,name,caltrans_district,fares_v2_status,fares_v2_status_array,n_published,fares_v2_distinct_status
0,City of Artesia,07 - Los Angeles,None,[],0,[]
1,City of Maywood,07 - Los Angeles,None,[],0,[]
2,City of Cerritos,07 - Los Angeles,None,[],0,[]
3,City of Shafter,06 - Fresno,None,[],0,[]
4,Colusa County Transit Agency,03 - Marysville,None,[],0,[]
...,...,...,...,...,...,...
184,Alameda-Contra Costa Transit District,04 - Oakland,"['Provider published', 'Vendor published', 'Provider published', 'Provider published']","[Provider published, Vendor published, Provider published, Provider published]",4,"[Provider published, Vendor published]"
185,San Joaquin Regional Transit District,10 - Stockton,"['Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish']","[Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish]",0,[Provider to Publish]
186,Eastern Sierra Transit Authority,09 - Bishop,"['Vendor published', 'Vendor published', 'Vendor published', 'Vendor published', 'Vendor published']","[Vendor published, Vendor published, Vendor published, Vendor published, Vendor published]",5,[Vendor published]
187,Sacramento Regional Transit District,03 - Marysville,"['Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish', 'Provider to Publish']","[Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish, Provider to Publish]",0,[Provider to Publish]


In [13]:
##check to see if the new column works
# (df>>filter(_.fares_v2_status_complete == 1)>>select(_.name,
#                                                      _.fares_v2_status,
#                                                     _.fares_v2_status_complete,
#                                                     _.fares_v2_status2))
## works

In [14]:
(_dla_utils.bar_chart_nosubset((df>>group_by(_.caltrans_district)>>count(_.reporting_category)),
                               'caltrans_district',
                               'n',
                               'reporting_category',
                               chart_title='Organizations Reporting Category by District'))

alt.Chart(...)

In [15]:
(_dla_utils.bar_chart_nosubset((df>>group_by(_.caltrans_district)>>count(_.organization_type)),
                               'caltrans_district',
                               'n',
                               'organization_type',
                               chart_title='Organizations Type by District'))

alt.Chart(...)

## What Percent of Organizations are GTFS Compliant by District?

In [16]:
(_dla_utils.bar_chart_nosubset(pct_gtfs,
                               'caltrans_district',
                               'org_count',
                               'caltrans_district',
                               chart_title='Number of Organizations by District'))

alt.Chart(...)

In [17]:
(_dla_utils.bar_chart_nosubset(pct_gtfs,
                               'caltrans_district',
                               'gtfs_static_count',
                               'caltrans_district',
                               chart_title='Number of GTFS Static Compliant Organizations by District'))

alt.Chart(...)

In [18]:
(_dla_utils.bar_chart_nosubset((pct_gtfs>>filter(_.caltrans_district.notnull())),
                               'caltrans_district',
                               'gtfs_rt_count',
                               'caltrans_district',
                               chart_title='Number of GTFS RT Compliant Organizations by District'))


alt.Chart(...)

In [19]:
(_dla_utils.bar_chart_nosubset(pct_gtfs,
                               'caltrans_district',
                               'percent_static',
                               'caltrans_district',
                               chart_title='Percent of GTFS Static Compliant Organizations by District'))

alt.Chart(...)

In [20]:
(_dla_utils.bar_chart_nosubset(pct_gtfs,
                               'caltrans_district',
                               'percent_rt',
                               'caltrans_district',
                               chart_title='Percent of GTFS Static Compliant Organizations by District'))

alt.Chart(...)

## Fares v2 Status by District

In [21]:
(display(Markdown(f"There are **{len((df>>filter(_.fares_v2_status_complete==1)))}** "
                  "Organizations with all Fares v2 Status as **fully published**")))

There are **73** Organizations with all Fares v2 Status as **fully published**

In [22]:
(_dla_utils.bar_chart_nosubset((df>>group_by(_.caltrans_district)>>count(_.fares_v2_status2)),
                               'caltrans_district',
                               'n',
                               'fares_v2_status2',
                               chart_title='Fares v2 Status by District'))

alt.Chart(...)